In [1]:
import requests
import json
import forecastio
from pprint import pprint
from geopy.geocoders import Nominatim
import pandas as pd
import import_ipynb
from SFO_airbnb import SFO_airbnb
import datetime as dt

importing Jupyter notebook from SFO_airbnb.ipynb


SFO_airbnb.ipynb:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  {
SFO_airbnb.ipynb:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  {
SFO_airbnb.ipynb:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  "outputs": [],


In [2]:
api_key = "a36c8345cc74c469a50078545e4970de"

In [9]:
geolocator = Nominatim(user_agent="ETL_Project")

location = geolocator.geocode("San Francisco")

SFO_coord = []
SFO_coord.append(location.latitude)
SFO_coord.append(location.longitude)

SFO_coord

[37.7792808, -122.4192363]

In [10]:
forecast = forecastio.load_forecast(api_key, SFO_coord[0], SFO_coord[1])

byDay = forecast.daily()
print(byDay.summary)
print(byDay.icon)
print(byDay.data[4].temperatureHigh)

Light rain today and Monday, with high temperatures rising to 58°F on Sunday.
rain
56.76


In [11]:
temp_high = []
temp_low = []
precip_perc = []
precip_type = []
precip_int = []
date = []

for i in byDay.data:
    date.append(i.time.date())
    temp_high.append(i.temperatureHigh)
    temp_low.append(i.temperatureLow)
    precip_perc.append(i.precipProbability)
    precip_int.append(i.precipIntensity)
    try:
        precip_type.append(i.precipType)
    except:
        precip_type.append("none")

date

dates = [dt.datetime.strftime(i, '%Y-%m-%d') for i in date]
dates

['2018-12-21',
 '2018-12-22',
 '2018-12-23',
 '2018-12-24',
 '2018-12-25',
 '2018-12-26',
 '2018-12-27',
 '2018-12-28']

In [12]:
df = pd.DataFrame({"Date": dates, 
                     "High Temp": temp_high, 
                     "Low Temp": temp_low, 
                     "Precipitation Probability": precip_perc,
                     "Precipitation Intensity": precip_int,
                     "Precipitation Type": precip_type
                    })
df.dtypes

Date                          object
High Temp                    float64
Low Temp                     float64
Precipitation Probability    float64
Precipitation Intensity      float64
Precipitation Type            object
dtype: object

In [13]:
SFO_airbnb.dtypes

Date         object
Under $50     int64
$50-100       int64
$100-150      int64
$150-200      int64
$200-250      int64
dtype: object

In [14]:
SFO_table = pd.merge(df, SFO_airbnb, on="Date")
SFO_table

,Date,High Temp,Low Temp,Precipitation Probability,Precipitation Intensity,Precipitation Type,Under $50,$50-100,$100-150,$150-200,$200-250
0,2018-12-21,56.46,47.96,0.56,0.0026,rain,177,799,798,470,248
1,2018-12-22,55.70,49.13,0.09,0.0003,rain,178,778,751,466,243
2,2018-12-23,57.97,54.17,0.18,0.0003,rain,192,792,751,457,255
3,2018-12-24,56.94,47.59,0.68,0.0219,rain,182,774,721,450,241
4,2018-12-25,56.76,47.56,0.09,0.0001,rain,188,763,724,463,241
5,2018-12-26,57.45,47.35,0.00,0.0000,none,191,779,755,477,235
6,2018-12-27,57.30,46.79,0.11,0.0002,rain,195,774,761,493,244
7,2018-12-28,56.52,45.49,0.00,0.0000,none,191,720,744,466,244
